In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')



# server = 'reporting-db.nystartfinans.net'
# database = 'reporting-db'
# username = 'Andreas'
# password = 'nCq8Sg@1lYnd(E'

server = 'reporting-db.nystartfinans.net'
database = 'reporting-db'
username = 'admin'
password = 'Tnb1tr9SNUgpJhQAc1lt'



driver = '{ODBC Driver 17 for SQL Server}'  # This is an example for SQL Server, adjust according to your database and installed ODBC driver



###############################   UC automatic data done    ############################### 


path = "../2. Code/Classes"
os.chdir(path)

from Preprocessing_Application_N import DataPreprocessor

processor = DataPreprocessor(server, database, username, password, driver)

# Relative path to the SQL file from the current directory (./2. Python/Classes)
relative_path_to_sql = "../../1. Data/Skuldsannering.sql"

# Use the relative path to fetch data
df = processor.fetch_data_from_sql(relative_path_to_sql)


print(max(df.DecisionDate))

UC_Accounts = df[df.AccountStatus.isin(['COLLECTION','FROZEN','OPEN'])]

UC_cleand = UC_Accounts[['AccountNumber','DisbursedDate','AccountStatus','EventTypeDesc','HasCoapp']]
UC_cleand['AccountNumber']  =  UC_cleand.AccountNumber.astype(int)
UC_cleand['Source']  = 'UC_Report'

UC_cleand = UC_cleand.drop_duplicates()



2024-04-01


In [2]:
import datetime

# Convert the string to a datetime.date object
cutoff_date = datetime.date(2024, 1, 1)

# Apply the filter using the datetime.date object
UC_cleand = UC_cleand[UC_cleand.DisbursedDate < cutoff_date]


In [3]:

###############################  2  Import OLD VILJA ACCOUNTS THAT WE CANNOT SEE IN DB   ############################### 



relative_path_to_excel = "../../1. Data/Skuldsanneringsdata Excel/Vilja Skuldsannering gammal.xlsx"

df = pd.read_excel(relative_path_to_excel, sheet_name="Vilja Gammal", engine='openpyxl')

df.rename(columns={'Lånekonto':'AccountNumber'},inplace=True)

df['AccountNumber']  =  df.AccountNumber.astype(int)



###############################  Find if main/CO   - I denna är alla med oavsett om vi inte kan se  om både huvud/medsök har skuldsannering  ############################### 



path = "../../1. Data/ApplicationsALL.sql"

appli = processor.fetch_data_from_sql(path)

appli['AccountNumber']  =  appli.AccountNumber.astype(int)




oldfull = pd.merge(df, appli,on='AccountNumber',how='left' )

oldfull = oldfull[oldfull.AccountStatus.isin(['COLLECTION','FROZEN','OPEN'])]


old_cleand = oldfull[['AccountNumber','DisbursedDate','AccountStatus','HasCoapp']].drop_duplicates()

old_cleand['EventTypeDesc'] = 'Skuldsanering bevilj'
old_cleand['Source'] = 'Vilja Old'
old_cleand = old_cleand.drop_duplicates()


# Filter to include rows where AccountNumber in old_cleand is not in AccountNumber in UC_cleand
old_cleand = old_cleand[~old_cleand.AccountNumber.isin(UC_cleand.AccountNumber)]


collect = pd.concat([UC_cleand,old_cleand])
collect = collect.drop_duplicates()


collect['LGD'] = np.where(collect.EventTypeDesc == 'Skuldsanering bevilj',0.5,1)

In [4]:
# Apply the filter using the datetime.date object
collect = collect[collect.DisbursedDate < cutoff_date]

In [5]:
today = datetime.date(2023, 12, 31)


In [6]:

collect['ReportingDate'] = today


collect = collect.drop_duplicates()
collect = collect.drop(columns=['HasCoapp','DisbursedDate','Source'])






In [7]:
collect

,AccountNumber,AccountStatus,EventTypeDesc,LGD,ReportingDate
0,5009758,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
1,5010517,COLLECTION,Personen har utvandr,1.0,2023-12-31
4,5012893,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
5,5014840,OPEN,Personen har utvandr,1.0,2023-12-31
6,5036348,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
...,...,...,...,...,...
178,7140890,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
182,7178353,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
187,7216278,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31
193,7289788,COLLECTION,Skuldsanering bevilj,0.5,2023-12-31


In [8]:

# Update the driver to 'ODBC Driver 17 for SQL Server' for Native Client 17
# This assumes you're using ODBC Driver 17, which is the usual driver used with SQL Server Native Client 17 installations
engine = sa.create_engine(f'mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server')

# Upload DataFrame to SQL table
collect.to_sql('SpecialCases', con=engine, index=False, if_exists='append', schema='nystart')



# excel_file_path = 'SpecialCases_mars_see.xlsx'

# # Export the DataFrame to Excel
# collect.to_excel(excel_file_path, index=False)


228